SETUP

In [1]:
import pandas as pd
import numpy as np
input_path = "D:/temp_files/datasets/customer_segmentation/Online Retail.xlsx"
scaled_output_path =  "D:/temp_files/datasets/customer_segmentation/freshman.xlsx"
raw_output_path = "D:/temp_files/datasets/customer_segmentation/raw_freshman.xlsx"
data = pd.read_excel(input_path)

Data Exploration

<https://baike.baidu.com/item/RFM%E6%A8%A1%E5%9E%8B/7070365?fr=aladdin>

<https://www.zhihu.com/question/505784526>

idx = list[np.where(data['CustomerID'].str.contains) 条件获取索引

查询df中数量小于0的行，提取index并转换成list

idx = data_temp.query('Quantity <= 0').index.tolist()

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


<https://www.kaggle.com/code/mohamedattia98/full-data-science-project/notebook>
* InvoiceNo invoice number
* StockCode	unique code for each transaction
* Description description of individual item
* Quantity number of items brought
* InvoiceDate data of purchase
* UnitPrice Price of individual unit
* CustomerID unique ID for each customer
* Country country name

https://archive.ics.uci.edu/ml/datasets/online+retail

缺失值以及无关行列的处理

In [3]:
# 去除ID列的nan值，在原数据集里表示为空白
data.dropna(subset='CustomerID', inplace=True)

In [4]:
# 去除无用列，暂时尝试不按照商品种类划分
features_droped = ['Description', 'StockCode', 'InvoiceNo']
data.drop(labels=features_droped, axis=1, inplace=True)

最近一次消费意指上一次购买的时候——顾客上一次是几时来店里、上一次根据哪本邮购目录购买东西、什么时候买的车，或在你的超市买早餐最近的一次是什么时候。

理论上，上一次消费时间越近的顾客应该是比较好的顾客，对提供即时的商品或是服务也最有可能会有反应。营销人员若想业绩有所成长，只能靠偷取竞争对手的市场占有率，而如果要密切地注意消费者的购买行为，那么最近的一次消费就是营销人员第一个要利用的工具。历史显示，如果我们能让消费者购买，他们就会持续购买。这也就是为什么，0至3个月的顾客收到营销人员的沟通信息多于3至6个月的顾客

消费的过程是持续变动的。在顾客距上一次购买时间满一个月之后，在数据库里就成为消费为两个月的客户。反之，同一天，消费为3个月前的客户作了其下一次的购买，他就成为消费为一天前的顾客，也就有可能在很短的期间内就收到新的折价信息。

recency
最近一次消费;时间;近因;近度;最近

In [5]:
# 基准时间的选取
last_datetime = data.InvoiceDate.max()

In [6]:
data_recency = data.groupby(['CustomerID']).InvoiceDate.agg([max])
def get_recency(x: np.datetime64):
    """
    param: x pd.Series中的某一行的值

    return: 计算np.datetime64的差值，再提取天数+1（原因出自RFM模型）
    """
    res_datetime = pd.to_datetime(last_datetime) - pd.to_datetime(x)
    return res_datetime.days + 1 # 最近一次是当天购买，算一天前刚买过
data_recency['recency'] = data_recency['max'].apply(get_recency)
data_recency.drop(axis=1, labels="max", inplace=True)

消费频率是顾客在限定的期间内所购买的次数。我们可以说最常购买的顾客，也是满意度最高的顾客。如果相信品牌及商店忠诚度的话，最常购买的消费者，忠诚度也就最高。增加顾客购买的次数意味着从竞争对手处偷取市场占有率，由别人的手中赚取营业额。

frequency

(声波或电磁波振动的)频率;频繁;发生率;出现率;重复率

In [7]:
# 获取统计时间内的最大时间、最小时间
max_datetime = data.InvoiceDate.max()
min_datetime = data.InvoiceDate.min()
# 获取时间间隔的天数形式
base_interval_datetime = pd.to_datetime(max_datetime) - pd.to_datetime(min_datetime)
base_interval_days = base_interval_datetime.days

In [8]:
# 按ID分组，获取日期的最大最小值和出现次数
data_frequency = data.groupby(['CustomerID']).InvoiceDate.agg([min, max, len])
# 在此标准下，计算光顾频率
data_frequency['frequency'] = data_frequency['len'].apply(lambda x: x / base_interval_days)
data_frequency.drop(labels=['min','max','len'], axis=1, inplace=True)

消费金额是所有数据库报告的支柱，也可以验证“帕雷托法则”(Pareto’s Law)——公司80%的收入来自20%的顾客。它显示出排名前10%的顾客所花费的金额比下一个等级者多出至少2倍，占公司所有营业额的40%以上。如看累计百分比的那一栏，我们会发现有40%的顾客贡献公司总营业额的80%；而有60%的客户占营业额的90%以上。最右的一栏显示每一等分顾客的平均消费，表现最好的 10%的顾客平均花费1195美元，而最差的10%仅有18美元 。

理论上M值和F值是一样的，都带有时间范围，指的是一段时间（通常是1年）内的消费金额，在工作中我认为对于一般店铺的类目而言，产品的价格带都是比较单一的，比如：同一品牌美妆类，价格浮动范围基本在某个特定消费群的可接受范围内，加上单一品类购买频次不高，所以对于一般店铺而言，M值对客户细分的作用相对较弱。

In [9]:
# 原数据列中增加total行，表示当前交易行为的总交易额
data['monetary'] = data.Quantity * data.UnitPrice
# 按ID分组后，算出该分组内的总额度
data_monetary = data.groupby(['CustomerID']).monetary.agg(sum)

现在尝试数据进行特征缩放:正态化的目的是让输出变得更加凸（球形数据让Kmean训练效果更佳）

In [10]:
data_rfm = pd.concat(objs=[data_recency, data_frequency, data_monetary], axis=1)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data_rfm = scaler.fit_transform(data_rfm)
scaled_data_rfm = pd.DataFrame(index=data_rfm.index,data=scaled_data_rfm, columns=data_rfm.columns)

代码输出到文件，从而保存预处理的结果，这不再是一个新手预处理，而是一个标准预处理

In [12]:
scaled_data_rfm.to_excel(scaled_output_path,sheet_name='Sheet1', index=True)
data_rfm.to_excel(raw_output_path, sheet_name='Sheet1', index=True)